In [75]:
import pandas as pd
import numpy as np

In [77]:
df = pd.read_csv('../data/GTspec_swissmod_query.protein_structure_info.just_glyco.csv').drop_duplicates()
df = df.dropna(subset=['struct_window_angstrom_resnum'])

In [78]:
proteins = []
aas = []
distances = []
for i in df.index:
    distance_and_aa_list = df.loc[i]['angstroms_to_feature'].split('|')
    aa_list = [da.split('_')[-1] for da in distance_and_aa_list]
    distance_list = [da.split('_')[0] for da in distance_and_aa_list]
    protein = [df.loc[i]['gene_id']] * len(distance_list)
    
    aas.extend(aa_list)
    distances.extend(distance_list)
    proteins.extend(protein)

In [86]:
merged = pd.concat([pd.Series(proteins),
           pd.Series(aas,dtype=int),
           pd.Series(distances,dtype=float)],
           axis=1)

In [89]:
merged.columns = ['protein','aa','distance_to_feature']
merged

,protein,aa,distance_to_feature
0,P31749,4,35.73
1,P31749,5,39.42
2,P31749,6,45.49
3,P31749,7,38.97
4,P31749,8,48.01
5,P31749,9,49.63
6,P31749,10,50.53
7,P31749,11,56.69
8,P31749,12,49.62
9,P31749,13,57.77


In [91]:
merged.drop_duplicates().to_csv('../data/protein_aa_distance.txt',sep='\t',header=None)

,protein,aa,distance_to_feature
0,P31749,4,35.73
1,P31749,5,39.42
2,P31749,6,45.49
3,P31749,7,38.97
4,P31749,8,48.01
5,P31749,9,49.63
6,P31749,10,50.53
7,P31749,11,56.69
8,P31749,12,49.62
9,P31749,13,57.77
